In [2]:
import sys
sys.path.insert(0,"../auto_agents/SegmentMap/")

from auto_agents.SegmentMap.AnyLoc.utilities import DinoV2ExtractFeatures
from auto_agents.SegmentMap import func
import os
import numpy as np
from natsort import natsorted
import cv2
import torch


Seed set to: 42 (type: <class 'int'>)


In [45]:
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans


In [3]:
dino = DinoV2ExtractFeatures("dinov2_vitg14", 31, 'value', device='cuda',norm_descs=False)


Using cache found in ./hub/facebookresearch_dinov2_main
/home/sourav/workspace/s/sg_habitat/./hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/sourav/workspace/s/sg_habitat/./hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/sourav/workspace/s/sg_habitat/./hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [ ]:
imgDir = "~/Downloads/VL-CMU-CD-binary255/train/"
imgDir_t0 = f'{imgDir}/t0/'
imgDir_t1 = f'{imgDir}/t1/'
imgDir_mask = f'{imgDir}/mask/'
imageNames = natsorted(os.listdir(imgDir_t0))

def name2Ft(imPath,dinoModel):
    img_p = cv2.imread(imPath)
    # plt.imshow(img_p)
    # plt.show()
    ft = func.getAnyLocFt(img_p, dinoModel,upsample=True)
    ft /= torch.linalg.norm(ft,axis=1,keepdims=True)
    return img_p, ft

for i,imgName in enumerate(imageNames):
    # if i==0:
    #     continue
    if imgName[-6:] != '_0.png':
        continue
    img0, ft0 = name2Ft(f'{imgDir_t0}/{imgName}',dino)
    img1, ft1 = name2Ft(f'{imgDir_t1}/{imgName}',dino)
    mask = cv2.imread(f'{imgDir_mask}/{imgName}',cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (ft0.shape[2], ft0.shape[3]), interpolation=cv2.INTER_NEAREST).astype(bool)
    labels = mask.reshape(-1)

    print(ft0.shape, ft1.shape)
    diff = (ft0 - ft1).squeeze().reshape([ft0.shape[1],-1]).T # n x d
    # diff /= torch.linalg.norm(diff,axis=1,keepdims=True)

    # print(diff.shape)
    pca = PCA(n_components=2)
    diff_pca = pca.fit_transform(diff.cpu().numpy())
    # print(diff_pca.shape)
    # plot diff_pca components and color each point through labels
    # plt.plot(diff_pca[labels,0],diff_pca[labels,1],'o',alpha=0.5)
    # plt.plot(diff_pca[~labels,0],diff_pca[~labels,1],'o',alpha=0.5)
    # plt.show()
    # cluster the pca features
    kmeans = KMeans(n_clusters=2, random_state=0).fit(diff_pca)
    # print(kmeans.labels_)
    # plt.plot(diff_pca[kmeans.labels_==0,0],diff_pca[kmeans.labels_==0,1],'o',alpha=0.5)
    # plt.plot(diff_pca[kmeans.labels_==1,0],diff_pca[kmeans.labels_==1,1],'o',alpha=0.5)
    # plt.show()

    # assign binary kmeans labels so that for the cluster with the most elements has label 1
    if (kmeans.labels_==1).sum() > (kmeans.labels_==0).sum():
        kmeans.labels_ = 1 - kmeans.labels_

    # plot the t0 and t1 image in top row
    # plot both the masks side by side with labels/titles
    plt.subplot(2,2,1)
    plt.imshow(img0[:,:,::-1])
    plt.title('t0')
    plt.subplot(2,2,2)
    plt.imshow(img1[:,:,::-1])
    plt.title('t1')
    plt.subplot(2,2,3)
    plt.imshow(mask)
    plt.title('GT Mask')
    plt.subplot(2,2,4)
    plt.imshow(kmeans.labels_.reshape(mask.shape))
    plt.title('PCA+Kmeans')
    plt.tight_layout()
    plt.axis('off')
    plt.savefig(f'../out/change/VL-CMU-CD-binary255/predMasks/{imgName}')
    plt.close()